In [30]:
import pandas as pd
import numpy as np
import pycountry
import csv
from mysql.connector import connect, Error
from getpass import getpass
from pathlib import Path


In [31]:
users_data = pd.read_csv("../Data/users-dataset.csv")

In [32]:
users_data["identifierHash"].duplicated().value_counts()

False    98913
Name: identifierHash, dtype: int64

In [33]:
columns_to_drop = [
    "country", "type", "civilityGenderId", "civilityTitle", 
    "seniorityAsMonths", "seniorityAsYears"
]
columns_to_rename = {
    "identifierHash": "customer_id",
    "index": "account_id",
    "socialNbFollowers": "followers",
    "socialNbFollows": "follows",
    "socialProductsLiked": "products_liked",
    "productsListed": "products_listed",
    "productsSold": "products_sold",
    "productsPassRate": "products_passrate",
    "productsWished": "products_wished",
    "productsBought": "products_bought",
    "hasAnyApp": "uses_any_app",
    "hasAndroidApp": "uses_android_app",
    "hasIosApp": "uses_ios_app",
    "hasProfilePicture": "profile_picture",
    "daysSinceLastLogin": "days_since_last_login",
    "seniority": "days_since_registration",
    "countryCode": "country_code"
}

users_data_v2 = users_data.reset_index().drop(columns_to_drop, axis=1).rename(columns = columns_to_rename)
users_data_v2.head(5)

,account_id,customer_id,language,followers,follows,products_liked,products_listed,products_sold,products_passrate,products_wished,products_bought,gender,uses_any_app,uses_android_app,uses_ios_app,profile_picture,days_since_last_login,days_since_registration,country_code
0,0,-1097895247965112460,en,147,10,77,26,174,74.0,104,1,M,True,False,True,True,11,3196,gb
1,1,2347567364561867620,en,167,8,2,19,170,99.0,0,0,F,True,False,True,True,12,3204,mc
2,2,6870940546848049750,fr,137,13,60,33,163,94.0,10,3,F,True,False,True,False,11,3203,fr
3,3,-4640272621319568052,en,131,10,14,122,152,92.0,7,0,F,True,False,True,False,12,3198,us
4,4,-5175830994878542658,en,167,8,0,25,125,100.0,0,0,F,False,False,False,True,22,2854,us


In [34]:
users_data_v2.dtypes

account_id                   int64
customer_id                  int64
language                    object
followers                    int64
follows                      int64
products_liked               int64
products_listed              int64
products_sold                int64
products_passrate          float64
products_wished              int64
products_bought              int64
gender                      object
uses_any_app                  bool
uses_android_app              bool
uses_ios_app                  bool
profile_picture               bool
days_since_last_login        int64
days_since_registration      int64
country_code                object
dtype: object

In [35]:
users_data_v2.nunique()

account_id                 98913
customer_id                98913
language                       5
followers                     90
follows                       85
products_liked               420
products_listed               65
products_sold                 75
products_passrate             72
products_wished              279
products_bought               70
gender                         2
uses_any_app                   2
uses_android_app               2
uses_ios_app                   2
profile_picture                2
days_since_last_login        699
days_since_registration       19
country_code                 199
dtype: int64

In [36]:
columns_to_inspect = [
    "language", "gender", "uses_any_app", "uses_android_app", "uses_ios_app", 
    "profile_picture"
]
users_data_v2.apply(lambda x: x.unique())

account_id                 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
customer_id                [-1097895247965112460, 2347567364561867620, 68...
language                                                [en, fr, de, it, es]
followers                  [147, 167, 137, 131, 130, 121, 53, 744, 57, 12...
follows                    [10, 8, 13, 12, 0, 9, 13764, 40, 19, 16, 60, 3...
products_liked             [77, 2, 60, 14, 0, 1, 1140, 3, 51671, 45, 863,...
products_listed            [26, 19, 33, 122, 25, 47, 31, 5, 0, 123, 40, 6...
products_sold              [174, 170, 163, 152, 125, 123, 108, 106, 104, ...
products_passrate          [74.0, 99.0, 94.0, 92.0, 100.0, 91.0, 98.0, 85...
products_wished            [104, 0, 10, 7, 531, 1842, 6, 68, 564, 2, 1016...
products_bought            [1, 0, 3, 105, 2, 36, 32, 14, 115, 6, 8, 69, 1...
gender                                                                [M, F]
uses_any_app                                                   [True, False]

In [37]:
for code in users_data_v2["country_code"]:
    try:
        country_name = pycountry.countries.get(alpha_2 = code).name
    except AttributeError:
        print(code)

ic
an
ic


In [38]:
values_to_drop = users_data_v2[users_data_v2["country_code"].isin(["ic", "an"])]
users_data_v3 = users_data_v2.drop(values_to_drop.index)

country = []
for code in users_data_v3["country_code"]:
    country_name = pycountry.countries.get(alpha_2 = code).name
    country.append(country_name)

users_data_v3["country"] = country
users_data_v3.head()

,account_id,customer_id,language,followers,follows,products_liked,products_listed,products_sold,products_passrate,products_wished,products_bought,gender,uses_any_app,uses_android_app,uses_ios_app,profile_picture,days_since_last_login,days_since_registration,country_code,country
0,0,-1097895247965112460,en,147,10,77,26,174,74.0,104,1,M,True,False,True,True,11,3196,gb,United Kingdom
1,1,2347567364561867620,en,167,8,2,19,170,99.0,0,0,F,True,False,True,True,12,3204,mc,Monaco
2,2,6870940546848049750,fr,137,13,60,33,163,94.0,10,3,F,True,False,True,False,11,3203,fr,France
3,3,-4640272621319568052,en,131,10,14,122,152,92.0,7,0,F,True,False,True,False,12,3198,us,United States
4,4,-5175830994878542658,en,167,8,0,25,125,100.0,0,0,F,False,False,False,True,22,2854,us,United States


In [39]:
country = users_data_v3.loc[:, ["country_code", "country"]].apply(lambda x: x.unique())
user_info = users_data_v3.loc[:, ["customer_id", "account_id", "language", "gender", "country_code"]]
user_account = users_data_v3.loc[:, ["account_id", "followers", "follows", "profile_picture"]].replace({True: 1, False: 0})
login_info = users_data_v3.loc[:, ["account_id", "days_since_last_login", "days_since_registration"]]
products = users_data_v3.loc[:, ["account_id", "products_liked", "products_listed", "products_sold", 
                                "products_passrate", "products_wished", "products_bought"]]
app_type = pd.DataFrame(["android", "ios"]).reset_index().rename(columns={"index": "app_id", 0: "app_type"})

to_melt = users_data_v3.loc[:, ["account_id", "uses_android_app", "uses_ios_app"]]
app = to_melt.melt(id_vars=["account_id"], value_vars=["uses_android_app", "uses_ios_app"], var_name="app_id", value_name="uses_app")
app = app.replace({"uses_android_app": 0, "uses_ios_app": 1, True: 1, False: 0})


In [55]:
p = Path("../DB/Db_Tables/ ")
p = str(p).strip()
tables_to_save = [country, user_account, user_info, login_info, products, app_type, app]
naming_tables = ["country", "user_account", "user_info", "login_info", "products", "app_type", "app"]
i = 0

for table in tables_to_save:
    table.to_csv(f"{p}{naming_tables[i]}.csv", index=False, header=False)
    i += 1

In [41]:
create_db_query = """
CREATE DATABASE vestiare_collective
"""

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print(e)

In [61]:
create_country_query = """
CREATE TABLE country(
    country_code VARCHAR(4) PRIMARY KEY,
    country VARCHAR(50)
);
"""

create_user_account_query = """
CREATE TABLE user_account(
    account_id MEDIUMINT PRIMARY KEY,
    followers SMALLINT,
    follows SMALLINT,
    profile_picture TINYINT
);
"""

create_user_info_query = """
CREATE TABLE user_info(
    customer_id BIGINT PRIMARY KEY,
    account_id MEDIUMINT,
    language VARCHAR(4),
    gender VARCHAR(1),
    country_code VARCHAR(4),
    FOREIGN KEY(country_code) REFERENCES country(country_code),
    FOREIGN KEY(account_id) REFERENCES user_account(account_id)
);
"""

create_app_type_query = """
CREATE TABLE app_type(
    app_id MEDIUMINT PRIMARY KEY,
    app_type VARCHAR(20)
);
"""

create_app_query = """
CREATE TABLE app(
    account_id MEDIUMINT,
    app_id MEDIUMINT,
    uses_app TINYINT,
    FOREIGN KEY(account_id) REFERENCES user_account(account_id),
    FOREIGN KEY(app_id) REFERENCES app_type(app_id),
    PRIMARY KEY(account_id, app_id)
);
"""

create_login_info_query = """
CREATE TABLE login_info(
    account_id MEDIUMINT,
    days_since_last_login SMALLINT,
    days_since_registration SMALLINT,
    FOREIGN KEY(account_id) REFERENCES user_account(account_id),
    PRIMARY KEY(account_id)
);
"""

create_products_query = """
CREATE TABLE products(
    account_id MEDIUMINT,
    products_liked MEDIUMINT,
    products_listed SMALLINT,
    products_sold SMALLINT,
    products_passrate FLOAT,
    products_wished SMALLINT,
    products_bought SMALLINT,
    FOREIGN KEY(account_id) REFERENCES user_account(account_id),
    PRIMARY KEY(account_id)
);
"""

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="vestiare_collective"
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(create_country_query)
            cursor.execute(create_user_account_query)
            cursor.execute(create_user_info_query)
            cursor.execute(create_app_type_query)
            cursor.execute(create_app_query)
            cursor.execute(create_login_info_query)
            cursor.execute(create_products_query)
            connection.commit()
except Error as e:
    print(e)

1050 (42S01): Table 'products' already exists


In [62]:
insert_country_query = """
INSERT INTO country
(country_code, country)
VALUES (%s, %s);
"""

insert_user_info_query = """
INSERT INTO user_info
(customer_id, account_id, language, gender, country_code)
VALUES (%s, %s, %s, %s, %s);
"""

insert_user_account_query = """
INSERT INTO user_account
(account_id, followers, follows, profile_picture)
VALUES (%s, %s, %s, %s);
"""

insert_app_type_query = """
INSERT INTO app_type
(app_id, app_type)
VALUES(%s, %s);
"""

insert_app_query = """
INSERT INTO app
(account_id, app_id, uses_app)
VALUES(%s, %s, %s);
"""

insert_login_info_query = """
INSERT INTO login_info
(account_id, days_since_last_login, days_since_registration)
VALUES(%s, %s, %s);
"""

insert_products_query = """
INSERT INTO products
(account_id, products_liked, products_listed, products_sold, 
products_passrate, products_wished, products_bought)
VALUES(%s, %s, %s, %s, %s, %s, %s);
"""

#Remember naming_tables
#naming_tables = ["country", "user_account", "user_info", "login_info", "products", "app_type", "app"]

queries_to_insert = [
    insert_country_query,
    insert_user_account_query,
    insert_user_info_query, 
    insert_login_info_query,
    insert_products_query,
    insert_app_type_query,
    insert_app_query
]

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="vestiare_collective"
    ) as connection:
            with connection.cursor() as cursor:
                i = 0
                for query in queries_to_insert:
                    with open(str(p).strip() + naming_tables[i] + ".csv") as file:
                        reader = csv.reader(file)
                        for row in reader:
                            cursor.execute(query, row)
                    i += 1
                connection.commit()
except Error as e:
    print(e)

In [63]:
query = """
SELECT *
FROM user_account;
"""

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="vestiare_collective"
    ) as connection:
            with connection.cursor() as cursor:
                cursor.execute(query)
                columns = cursor.description
                data = cursor.fetchall()
except Error as e:
    print(e)

column_names = []
for column in columns:
    column_names.append(column[0])
pd.DataFrame(data, columns=column_names)

,account_id,followers,follows,profile_picture
0,0,147,10,1
1,1,167,8,1
2,2,137,13,0
3,3,131,10,0
4,4,167,8,1
...,...,...,...,...
98905,98908,3,8,1
98906,98909,3,8,1
98907,98910,3,8,1
98908,98911,3,8,1


In [64]:
query = """
SELECT
user_account.account_id,
login_info.days_since_last_login,
login_info.days_since_registration,
user_info.gender,
user_info.country_code,
user_account.followers,
user_account.follows,
user_account.profile_picture,
products.products_liked,
products.products_listed,
products.products_sold,
products.products_passrate,
products.products_wished,
products.products_bought

FROM user_account
LEFT JOIN user_info ON user_account.account_id = user_info.account_id
LEFT JOIN login_info ON user_account.account_id = login_info.account_id
LEFT JOIN products ON user_account.account_id = products.account_id
"""

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="vestiare_collective"
    ) as connection:
            with connection.cursor() as cursor:
                cursor.execute(query)
                columns = cursor.description
                data = cursor.fetchall()
except Error as e:
    print(e)

column_names = []
for column in columns:
    column_names.append(column[0])
df = pd.DataFrame(data, columns=column_names)

In [65]:
query = """
SELECT account_id, app.uses_app, app_type.app_type
FROM app
JOIN app_type ON app.app_id = app_type.app_id
"""

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="vestiare_collective"
    ) as connection:
            with connection.cursor() as cursor:
                cursor.execute(query)
                columns = cursor.description
                data = cursor.fetchall()
except Error as e:
    print(e)

column_names = []
for column in columns:
    column_names.append(column[0])
to_pivot = pd.DataFrame(data, columns=column_names)

In [66]:
to_merge = to_pivot.pivot(columns="app_type", index="account_id", values="uses_app").reset_index()
df = df.merge(to_merge, on="account_id")
df

,account_id,days_since_last_login,days_since_registration,gender,country_code,followers,follows,profile_picture,products_liked,products_listed,products_sold,products_passrate,products_wished,products_bought,android,ios
0,0,11,3196,M,gb,147,10,1,77,26,174,74.0,104,1,0,1
1,1,12,3204,F,mc,167,8,1,2,19,170,99.0,0,0,0,1
2,2,11,3203,F,fr,137,13,0,60,33,163,94.0,10,3,0,1
3,3,12,3198,F,us,131,10,0,14,122,152,92.0,7,0,0,1
4,4,22,2854,F,us,167,8,1,0,25,125,100.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98905,98908,708,3204,M,us,3,8,1,0,0,0,0.0,0,0,0,0
98906,98909,695,3204,M,fr,3,8,1,0,0,0,0.0,0,0,0,1
98907,98910,520,3204,M,be,3,8,1,0,0,0,0.0,0,0,1,0
98908,98911,267,3204,F,it,3,8,1,0,0,0,0.0,0,0,0,0


In [67]:
p = Path("../Tableau_Data/ ")
df.to_csv(f"{p}".strip() + "user_retention_data.csv", index=False)

In [68]:
df

,account_id,days_since_last_login,days_since_registration,gender,country_code,followers,follows,profile_picture,products_liked,products_listed,products_sold,products_passrate,products_wished,products_bought,android,ios
0,0,11,3196,M,gb,147,10,1,77,26,174,74.0,104,1,0,1
1,1,12,3204,F,mc,167,8,1,2,19,170,99.0,0,0,0,1
2,2,11,3203,F,fr,137,13,0,60,33,163,94.0,10,3,0,1
3,3,12,3198,F,us,131,10,0,14,122,152,92.0,7,0,0,1
4,4,22,2854,F,us,167,8,1,0,25,125,100.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98905,98908,708,3204,M,us,3,8,1,0,0,0,0.0,0,0,0,0
98906,98909,695,3204,M,fr,3,8,1,0,0,0,0.0,0,0,0,1
98907,98910,520,3204,M,be,3,8,1,0,0,0,0.0,0,0,1,0
98908,98911,267,3204,F,it,3,8,1,0,0,0,0.0,0,0,0,0


![one_year](https://raw.githubusercontent.com/Matagon/wip/main/Vestiaire_Collective/Tableau_Analysis/one_year.png?token=GHSAT0AAAAAABVGARERXMMATRREZFJ2NAT2YUY5X5A)

![short](https://raw.githubusercontent.com/Matagon/wip/main/Vestiaire_Collective/Tableau_Analysis/short.png?token=GHSAT0AAAAAABVGAREQU3NUAFB75WO3N4P2YUY46VQ)

![long](https://raw.githubusercontent.com/Matagon/wip/main/Vestiaire_Collective/Tableau_Analysis/long.png?token=GHSAT0AAAAAABVGAREQGA2BZS26WY3RV3UKYUY47CQ)

![default_accounts](https://raw.githubusercontent.com/Matagon/wip/main/Vestiaire_Collective/Tableau_Analysis/default_accounts.png?token=GHSAT0AAAAAABVGAREQXUPWERJSOHTRNZNSYUY47QQ)

![buyer_overview](https://raw.githubusercontent.com/Matagon/wip/main/Vestiaire_Collective/Tableau_Analysis/buyer_overview.png?token=GHSAT0AAAAAABVGAREQJ4JWBLJ3SXARL522YUY475Q)

![buyer_products_liked_comparison](https://raw.githubusercontent.com/Matagon/wip/main/Vestiaire_Collective/Tableau_Analysis/buyer_products_liked_comparison.png?token=GHSAT0AAAAAABVGAREQ4YAUFQIFXXGLA5L6YUY5AJA)

![buyer_products_wished_comparison](https://raw.githubusercontent.com/Matagon/wip/main/Vestiaire_Collective/Tableau_Analysis/buyer_products_wished_comparison.png?token=GHSAT0AAAAAABVGAREQXHNPKRHS7SU6KDSSYUY5AVQ)

![buyer_products_bought_comparison](https://raw.githubusercontent.com/Matagon/wip/main/Vestiaire_Collective/Tableau_Analysis/buyer_products_bought_comparison.png?token=GHSAT0AAAAAABVGAREROHRFYIP7FSNPIO4EYUY5BCQ)